In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn import preprocessing, svm  
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model, metrics
from sklearn.svm import SVC  
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.feature_selection import chi2,SelectKBest,f_classif,mutual_info_classif

In [2]:
lung_cancer=pd.read_csv("lungcancer2.csv")
lung_cancer.head()

C:\Users\babu saheb sah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Patient ID,Age recode with <1 year olds,Sex,Grade,Total number of in situ/malignant tumors for patient,T value - based on AJCC 3rd (1988-2003),N value - based on AJCC 3rd (1988-2003),M value - based on AJCC 3rd (1988-2003),Primary Site - labeled,Primary Site,Type of Reporting Source,First malignant primary indicator,Sequence number,CS lymph nodes (2004-2015),Histology recode - broad groupings,RX Summ--Scope Reg LN Sur (2003+),RX Summ--Surg Prim Site (1998+),CS tumor size (2004-2015),Survival months
0,273,75-79 years,Female,Well differentiated; Grade I,2,Blank(s),Blank(s),Blank(s),"C34.3-Lower lobe, lung",343,Hospital inpatient/outpatient or clinic,No,2nd of 2 or more primaries,0,8140-8389: adenomas and adenocarcinomas,4 or more regional lymph nodes removed,30,31,128
1,390,60-64 years,Male,Poorly differentiated; Grade III,2,T4,N3,M0,C34.0-Main bronchus,340,Hospital inpatient/outpatient or clinic,No,2nd of 2 or more primaries,Blank(s),8140-8389: adenomas and adenocarcinomas,Blank(s),0,Blank(s),1
2,455,75-79 years,Male,Poorly differentiated; Grade III,2,Blank(s),Blank(s),Blank(s),"C34.3-Lower lobe, lung",343,Hospital inpatient/outpatient or clinic,No,2nd of 2 or more primaries,0,8050-8089: squamous cell neoplasms,4 or more regional lymph nodes removed,30,55,5
3,542,75-79 years,Female,Undifferentiated; anaplastic; Grade IV,2,TX,N3,M1,"C34.1-Upper lobe, lung",341,Hospital inpatient/outpatient or clinic,No,2nd of 2 or more primaries,Blank(s),"8010-8049: epithelial neoplasms, NOS",Blank(s),0,Blank(s),1
4,565,70-74 years,Female,Unknown,2,Blank(s),Blank(s),Blank(s),"C34.1-Upper lobe, lung",341,Hospital inpatient/outpatient or clinic,No,2nd of 2 or more primaries,0,"8010-8049: epithelial neoplasms, NOS",None,0,44,0


In [3]:
lung_cancer = lung_cancer.replace({'Blank(s)':np.nan})

In [4]:
lung_cancer.rename(columns = {'T value - based on AJCC 3rd (1988-2003)':'T_value'}, inplace = True)
lung_cancer.rename(columns = {'N value - based on AJCC 3rd (1988-2003)':'N_value'}, inplace = True)
lung_cancer.rename(columns = {'M value - based on AJCC 3rd (1988-2003)':'M_value'}, inplace = True)
lung_cancer.rename(columns = { 'Survival months':'Survival_months'}, inplace = True)
lung_cancer.rename(columns = {'Age recode with <1 year olds':'Age'}, inplace = True)
lung_cancer.rename(columns = {'Total number of in situ/malignant tumors for patient':'NumMaligantTumor'}, inplace = True)
lung_cancer.rename(columns = {'Type of Reporting Source':'ReportingSource'}, inplace = True)
lung_cancer.rename(columns = {'First malignant primary indicator':'MaligantIndicator'}, inplace = True)
lung_cancer.rename(columns = {'CS lymph nodes (2004-2015)':'CSLymphNodes'}, inplace = True)
lung_cancer.rename(columns = {'Histology recode - broad groupings':'History'}, inplace = True)
lung_cancer.rename(columns = {'RX Summ--Surg Prim Site (1998+)':'RX Summ--Surg Prim Site'}, inplace = True)
lung_cancer.rename(columns = {'CS tumor size (2004-2015)':'CSTumorSize'}, inplace = True)
lung_cancer.rename(columns = {'Primary Site - labeled':'PrimarySite'}, inplace = True)

lung_cancer.rename(columns = {'RX Summ--Scope Reg LN Sur (2003+)':'RX_Summ_Scope_Reg_LN_Sur'}, inplace = True)

In [5]:
lung_cancer.isna().sum()

Patient ID                       0
Age                              0
Sex                              0
Grade                            0
NumMaligantTumor                 0
T_value                     722421
N_value                     722421
M_value                     722421
PrimarySite                      0
Primary Site                     0
ReportingSource                  0
MaligantIndicator                0
Sequence number                  0
CSLymphNodes                302472
History                          0
RX_Summ_Scope_Reg_LN_Sur    147973
RX Summ--Surg Prim Site          0
CSTumorSize                 302472
Survival_months                  0
dtype: int64

In [6]:
for col in lung_cancer.columns:
    print('{} :unique {}'.format(col, lung_cancer[col].unique()))
    print('****************************************************')

Patient ID :unique [     273      390      455 ... 75826061 75826072 75826108]
****************************************************
Age :unique ['75-79 years' '60-64 years' '70-74 years' '65-69 years' '50-54 years'
 '55-59 years' '80-84 years' '85+ years' '45-49 years' '25-29 years'
 '35-39 years' '40-44 years' '30-34 years' '20-24 years' '15-19 years'
 '10-14 years' '05-09 years' '00 years' '01-04 years']
****************************************************
Sex :unique ['Female' 'Male']
****************************************************
Grade :unique ['Well differentiated; Grade I' 'Poorly differentiated; Grade III'
 'Undifferentiated; anaplastic; Grade IV' 'Unknown'
 'Moderately differentiated; Grade II']
****************************************************
NumMaligantTumor :unique [2 3 4 1 5 6 7 9 8 '1' '2' '3' '4' '6' '5' '7' '8' '9' 'Unknown' 11 14 10
 '11' 34 '10' 15 '20']
****************************************************
T_value :unique [nan 'T4' 'TX' 'T1' 'TXa' 'T2' 'TXc' 

In [7]:
lung_cancer = lung_cancer.drop(columns=['T_value','N_value','M_value', 'ReportingSource','Primary Site'])

In [8]:
lung_cancer.isna().sum()

Patient ID                       0
Age                              0
Sex                              0
Grade                            0
NumMaligantTumor                 0
PrimarySite                      0
MaligantIndicator                0
Sequence number                  0
CSLymphNodes                302472
History                          0
RX_Summ_Scope_Reg_LN_Sur    147973
RX Summ--Surg Prim Site          0
CSTumorSize                 302472
Survival_months                  0
dtype: int64

In [11]:
lung_cancer = lung_cancer[lung_cancer['CSTumorSize'].notna()]
lung_cancer = lung_cancer.reset_index()

In [12]:
lung_cancer = lung_cancer.drop(columns=['index','Patient ID'])

In [13]:
lung_cancer = lung_cancer.drop(lung_cancer[(lung_cancer.Survival_months =='Unknown')].index)

In [15]:
lung_cancer = lung_cancer.drop(lung_cancer[(lung_cancer.NumMaligantTumor =='Unknown')].index)

In [16]:
lung_cancer = lung_cancer.drop(lung_cancer[(lung_cancer.Grade =='Unknown')].index)

In [17]:
lung_cancer = lung_cancer.reset_index()
lung_cancer = lung_cancer.drop(columns=['index'])

In [18]:
for col in lung_cancer.columns:
    print('{} :unique {}'.format(col, lung_cancer[col].unique()))
    print('****************************************************')

Age :unique ['75-79 years' '50-54 years' '80-84 years' '60-64 years' '70-74 years'
 '55-59 years' '25-29 years' '85+ years' '65-69 years' '45-49 years'
 '40-44 years' '35-39 years' '10-14 years' '15-19 years' '20-24 years'
 '30-34 years' '05-09 years' '01-04 years' '00 years']
****************************************************
Sex :unique ['Female' 'Male']
****************************************************
Grade :unique ['Well differentiated; Grade I' 'Poorly differentiated; Grade III'
 'Undifferentiated; anaplastic; Grade IV'
 'Moderately differentiated; Grade II']
****************************************************
NumMaligantTumor :unique [2 4 3 5 6 7 9 8 1 '1' '2' '3' '4' '6' '5' '7' '8' '9' 11 '11' 10 15 '20'
 '10']
****************************************************
PrimarySite :unique ['C34.3-Lower lobe, lung' 'C34.0-Main bronchus' 'C34.1-Upper lobe, lung'
 'C34.2-Middle lobe, lung' 'C34.9-Lung, NOS'
 'C34.8-Overlapping lesion of lung']
************************************

In [19]:
lung_cancer.dtypes

Age                         object
Sex                         object
Grade                       object
NumMaligantTumor            object
PrimarySite                 object
MaligantIndicator           object
Sequence number             object
CSLymphNodes                object
History                     object
RX_Summ_Scope_Reg_LN_Sur    object
RX Summ--Surg Prim Site      int64
CSTumorSize                 object
Survival_months             object
dtype: object

In [20]:
to_transform = ['Age', 'Sex','Grade', 'PrimarySite',
                'MaligantIndicator', 'Sequence number',  'History',
                'RX_Summ_Scope_Reg_LN_Sur']

In [21]:
for col in lung_cancer.columns:
    print(col)
    if col in to_transform:
        lung_cancer[col] = pd.Categorical(lung_cancer[col])
    else:
        lung_cancer[col] = pd.to_numeric(lung_cancer[col])

Age
Sex
Grade
NumMaligantTumor
PrimarySite
MaligantIndicator
Sequence number
CSLymphNodes
History
RX_Summ_Scope_Reg_LN_Sur
RX Summ--Surg Prim Site
CSTumorSize
Survival_months


In [22]:
lung_cancer.dtypes

Age                         category
Sex                         category
Grade                       category
NumMaligantTumor               int64
PrimarySite                 category
MaligantIndicator           category
Sequence number             category
CSLymphNodes                   int64
History                     category
RX_Summ_Scope_Reg_LN_Sur    category
RX Summ--Surg Prim Site        int64
CSTumorSize                    int64
Survival_months                int64
dtype: object

In [23]:
df = lung_cancer.copy()

In [24]:
le = preprocessing.LabelEncoder()

In [25]:
for col in to_transform:
    df[col] = le.fit_transform(df[col])

In [26]:
df=df[df['Survival_months']<=35]
#df=df[df['Survival_months']<=72]

In [27]:
x = df[df.columns[:-1]].to_numpy()
y = df[df.columns[-1]].to_numpy()

In [28]:
### chi2 test
test2 = SelectKBest(chi2, k=9)
x_new= test2.fit_transform(x, y)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
import math

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=101)

In [31]:
print("True mean value",np.mean(y_train))
print("True Standard Deviation value",np.std(y_train))

True mean value 10.822138286411484
True Standard Deviation value 10.018077297196946


In [34]:
s=[]
rf = RandomForestRegressor(n_estimators = 100 ,random_state = 42)
cv = KFold(n_splits=5, random_state=40, shuffle=True)
for train_index, test_index in cv.split(x_train):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    x1_train, x1_test, y1_train, y1_test = x_train[train_index], x_train[test_index], y_train[train_index], y_train[test_index]
    rf.fit(x1_train, y1_train)
    s.append(rf.score(x1_test, y1_test))

Train Index:  [     0      1      3 ... 154082 154084 154085] 

Test Index:  [     2      4      5 ... 154073 154076 154083]
Train Index:  [     0      2      3 ... 154082 154083 154084] 

Test Index:  [     1      6      9 ... 154078 154079 154085]
Train Index:  [     0      1      2 ... 154082 154083 154085] 

Test Index:  [     3     10     14 ... 154075 154080 154084]
Train Index:  [     0      1      2 ... 154083 154084 154085] 

Test Index:  [     7     23     25 ... 154069 154081 154082]
Train Index:  [     1      2      3 ... 154083 154084 154085] 

Test Index:  [     0      8     21 ... 154064 154071 154074]


In [35]:
# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 100,random_state = 42)
# Train the model on training data
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [36]:
# Predict the final value
prediction1=rf.predict(x_test)

In [37]:
# find different metrices
final_value=abs(y_test-prediction1)
print("Mean Value",np.mean(prediction1))
print("Standard Deviation Value",np.std(prediction1))
print("standard Deviation residual",np.std(final_value))

Mean Value 11.0096096068834
Standard Deviation Value 6.113904462585466
standard Deviation residual 5.837843799573205


In [38]:
# find the mean square error
error=mean_squared_error(y_test,prediction1)
print('Root Mean Squared Error:',round(math.sqrt(error),2))

Root Mean Squared Error: 9.54


In [39]:
#Linear Regression
from sklearn.linear_model import LogisticRegression

In [41]:
scores=[]
log_reg=LinearRegression()
cv = KFold(n_splits=5, random_state=40, shuffle=True)
for train_index, test_index in cv.split(x_train):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    x1_train, x1_test, y1_train, y1_test = x_train[train_index], x_train[test_index], y_train[train_index], y_train[test_index]
    log_reg.fit(x1_train, y1_train)
    scores.append(log_reg.score(x1_test, y1_test))

Train Index:  [     0      1      3 ... 154082 154084 154085] 

Test Index:  [     2      4      5 ... 154073 154076 154083]
Train Index:  [     0      2      3 ... 154082 154083 154084] 

Test Index:  [     1      6      9 ... 154078 154079 154085]
Train Index:  [     0      1      2 ... 154082 154083 154085] 

Test Index:  [     3     10     14 ... 154075 154080 154084]
Train Index:  [     0      1      2 ... 154083 154084 154085] 

Test Index:  [     7     23     25 ... 154069 154081 154082]
Train Index:  [     1      2      3 ... 154083 154084 154085] 

Test Index:  [     0      8     21 ... 154064 154071 154074]


In [43]:
### Trained the model with best params 
log_reg=LinearRegression()
log_reg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [44]:
# Predict the final value
prediction2=log_reg.predict(x_test)

In [45]:
# Find different Metrices
final_value=abs(y_test-prediction2)
print("Mean Value",np.mean(prediction2))
print("Standard Deviation Value",np.std(prediction2))
print("standard Deviation residual",np.std(final_value))

Mean Value 10.80031505456779
Standard Deviation Value 4.102879383480804
standard Deviation residual 5.264996068251592


In [46]:
# find mean square error
error=mean_squared_error(y_test,prediction2)
print('Root Mean Squared Error:',round(math.sqrt(error),2))

Root Mean Squared Error: 9.16
